In [1]:
import json
import pandas as pd
import firebase_admin
from datetime import datetime
from firebase_admin import credentials
from firebase_admin import firestore

In [2]:

df = pd.read_csv('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Text Normalization\\Tweets.csv')

In [3]:
#df.dropna(inplace= True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4791 entries, 0 to 4790
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  4791 non-null   int64 
 1   Tweet       4791 non-null   object
 2   Location    4791 non-null   object
 3   year        4791 non-null   int64 
 4   month       4791 non-null   int64 
 5   day         4791 non-null   int64 
 6   Sentiment   4791 non-null   object
dtypes: int64(4), object(3)
memory usage: 262.1+ KB


In [4]:
CredNew = credentials.Certificate('C:\\Users\\fmabd\\Documents\\GitHub\\Smart-Customer-Care\\Database\\smartcc-960f7-firebase-adminsdk-ijo0i-27aa89d85e.json')
new=firebase_admin.initialize_app(CredNew,name="new")

db = firestore.client(new)

In [5]:
csv= json.loads(json.dumps(df[0:2500].to_dict()))
csv2= json.loads(json.dumps(df[2500:].to_dict()))

In [6]:
def UploadDate (data1, data2):
    now = datetime.now()
    now = str(now) 
    tweets= db.collection(u'Twitter Data').document(now).set(data1)
    
    now1 = datetime.now()
    now1 = str(now1)
    tweets1= db.collection(u'Twitter Data').document(now1).set(data2)

In [7]:
UploadDate(csv,csv2)